In [ ]:
## Use Bigquery to automatically process GWAS with sumstats.

import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine

from google.cloud import bigquery

client = bigquery.Client()

# # Perform a query.
# QUERY = (
#     'WITH
#   SNP_info AS (
#   SELECT
#     CONCAT(CAST(chrom AS string), CAST(pos AS string), CAST(ref AS string), CAST(alt AS string)) AS identifier,
#     ref,
#     alt,
#     n_total,
#     pval,
#     eaf
#   FROM
#     `open-targets-genetics-dev.Jack_sumstats.GCST006867` )
# SELECT
#   *
# FROM
#   SNP_info
# JOIN (
#   SELECT
#     CONCAT(CAST(chr_id AS string), CAST(position AS string), CAST(ref_allele AS string), CAST(alt_allele AS string)) AS identifier,
#     rs_id
#   FROM
#     `open-targets-genetics.210608.variants` ) variants
# ON
#   SNP_info.identifier = variants.identifier')

# Perform a query.
QUERY = (
    'SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` '
    'WHERE state = "TX" '
    'LIMIT 100')
    
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.name)


In [8]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client.from_service_account_json("/Users/xg1/Downloads/open-targets-genetics-eaa50bdaa2f5.json")

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)


In [ ]:
implicit()

In [14]:
Latest_study_table=pd.read_parquet("/Users/xg1/Downloads/v2d_210922_studies.parquet")
Latest_sumstats=Latest_study_table.loc[Latest_study_table["has_sumstats"] == True]
Latest_sumstats

,study_id,pmid,pub_date,pub_journal,pub_title,pub_author,trait_reported,trait_efos,ancestry_initial,ancestry_replication,n_initial,n_replication,n_cases,trait_category,num_assoc_loci,has_sumstats
0,FINNGEN_R5_AB1_AMOEBIASIS,,2021-5-11,,,FINNGEN_R5,Amoebiasis,[EFO_0007144],[European=197385],[],197385,0.0,87.0,Disease,0,True
1,FINNGEN_R5_AB1_ANOGENITAL_HERPES_SIMPLEX,,2021-5-11,,,FINNGEN_R5,Anogenital herpesviral [herpes simplex] infection,[EFO_0007282],[European=213843],[],213843,0.0,891.0,Disease of genitourinary system,0,True
2,FINNGEN_R5_AB1_ARTHROPOD,,2021-5-11,,,FINNGEN_R5,Arthropod-borne viral fevers and viral haemorr...,"[EFO_0000763, MONDO_0018087]",[European=218792],[],218792,0.0,1450.0,Disease,0,True
3,FINNGEN_R5_AB1_ASPERGILLOSIS,,2021-5-11,,,FINNGEN_R5,Aspergillosis,[EFO_0007157],[European=214903],[],214903,0.0,87.0,Disease,0,True
4,FINNGEN_R5_AB1_ATYPICAL_CNS_VIRUS,,2021-5-11,,,FINNGEN_R5,Atypical virus infections of central nervous s...,[MONDO_0024318],[European=217783],[],217783,0.0,146.0,Uncategorised,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36219,SAIGE_988,,2018-10-24,,,UKB SAIGE,Toxic effect of noxious substances eaten as food,[EFO_0008546],[European=408481],[],408481,0.0,114.0,Disease,0,True
36220,SAIGE_989,,2018-10-24,,,UKB SAIGE,"Toxic effect of other substances, chiefly nonm...",[EFO_0008546],[European=408425],[],408425,0.0,58.0,Disease,0,True
36221,SAIGE_990,,2018-10-24,,,UKB SAIGE,Effects radiation nos,[EFO_0009658],[European=406425],[],406425,0.0,3130.0,Disease,0,True
36222,SAIGE_994,,2018-10-24,,,UKB SAIGE,Sepsis and sirs,[HP_0100806],[European=408961],[],408961,0.0,2811.0,Uncategorised,0,True


In [17]:
import os
list_sumstats="bq.py --help"
os.system(list_sumstats)

32512

In [13]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "open-targets-genetics-dev.Jack_sumstats."+"your_table_name"

# job_config = bigquery.LoadJobConfig(
#     schema=[
#         bigquery.SchemaField("name", "STRING"),
#         bigquery.SchemaField("post_abbr", "STRING"),
#     ],
#     skip_leading_rows=1,
#     # The source format defaults to CSV, so the line below is optional.
#     source_format=bigquery.SourceFormat.CSV,
# )

job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET,)

uri = "gs://genetics-portal-dev-sumstats/unfiltered/gwas/GCST000568.parquet/*.parquet"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 2445642 rows.


In [17]:
 # Make an API request.




In [18]:
query_job.result()

In [5]:
from google.cloud import bigquery
client = bigquery.Client()

Input_study="GCST000568"



## LOAD Job: load from GCS to BQ (table_id)
    # Would it be possible to make this table temporary? Or delete itself automatically after 1 week?

Input_sumstats_path="gs://genetics-portal-dev-sumstats/unfiltered/gwas"
Input_study_URI=Input_sumstats_path+"/"+Input_study+".parquet/*.parquet"

table_id = "open-targets-genetics-dev.Jack_sumstats."+Input_study

load_job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET,)
load_job = client.load_table_from_uri(
    Input_study_URI, table_id, job_config=load_job_config
)  # Make an API request.
load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

# Query Job

client = bigquery.Client()
rsID_table="open-targets-genetics-dev.rsID_sumstats."+Input_study

query_job_config = bigquery.QueryJobConfig(destination=rsID_table)
query = """
    WITH SNP_info AS (
    SELECT
        CONCAT(CAST(chrom AS string), CAST(pos AS string), CAST(ref AS string), CAST(alt AS string)) AS identifier,
        ref,
        alt,
        n_total,
        pval,
        eaf,
        beta
    FROM
        `{0}` )
    SELECT
    rs_id AS RSID, ref AS A1, alt AS A2, n_total AS N, pval AS P, eaf AS EAF, beta AS BETA
    FROM
    SNP_info
    JOIN (
    SELECT
        CONCAT(CAST(chr_id AS string), CAST(position AS string), CAST(ref_allele AS string), CAST(alt_allele AS string)) AS identifier,
        rs_id
    FROM
        `open-targets-genetics.210608.variants` ) variants
    USING(identifier)
""".format(table_id)

query_job = client.query(query, job_config=query_job_config)
query_job.result()

rsID_GCS_bucket="gs://genetics-portal-dev-analysis/xg1/rsid_sumstats"
rsID_GCS_URI=rsID_GCS_bucket+"/{0}.txt".format(Input_study)

extract_job_config = bigquery.ExtractJobConfig() 
extract_job_config.field_delimiter = '\t'
extract_job_config.compression='GZIP'

extract_job = client.extract_table(
    rsID_table,
    rsID_GCS_URI,
    # Location must match that of the source table.
    location="EU",
    job_config=extract_job_config
)  # API request
extract_job.result()  # Waits for job to complete.

print(
    "Exported {} to {}".format(rsID_table, rsID_GCS_URI)
)


#print("The query data:")
#for row in query_job:
    # Row values can be accessed by field name or index.
    #print("name={}, count={}".format(row[0], row["total_people"]))


Loaded 12228210 rows.
Exported open-targets-genetics-dev.rsID_sumstats.GCST000568 to gs://genetics-portal-dev-analysis/xg1/rsid_sumstats/GCST000568


In [7]:
from google.cloud import bigquery
client = bigquery.Client()

#Input_study="GCST000568"
Input_study=["GCST005413", "GCST004131", "GCST005569", "GCST002245"]
## LOAD Job: load from GCS to BQ (table_id)
    # Would it be possible to make this table temporary? Or delete itself automatically after 1 week?

for study in Input_study:
    rsID_table="open-targets-genetics-dev.rsID_sumstats."+study

    rsID_GCS_bucket="gs://genetics-portal-dev-analysis/xg1/rsid_sumstats"
    rsID_GCS_URI=rsID_GCS_bucket+"/{0}.txt.gz".format(study)

    extract_job_config = bigquery.ExtractJobConfig() 
    extract_job_config.field_delimiter = '\t'
    extract_job_config.compression='GZIP'

    extract_job = client.extract_table(
        rsID_table,
        rsID_GCS_URI,
        # Location must match that of the source table.
        location="EU",
        job_config=extract_job_config
    )  # API request
    extract_job.result()  # Waits for job to complete.

    print(
        "Exported {} to {}".format(rsID_table, rsID_GCS_URI)
    )


Exported open-targets-genetics-dev.rsID_sumstats.GCST005413 to gs://genetics-portal-dev-analysis/xg1/rsid_sumstats/GCST005413.txt.gz
Exported open-targets-genetics-dev.rsID_sumstats.GCST004131 to gs://genetics-portal-dev-analysis/xg1/rsid_sumstats/GCST004131.txt.gz
Exported open-targets-genetics-dev.rsID_sumstats.GCST005569 to gs://genetics-portal-dev-analysis/xg1/rsid_sumstats/GCST005569.txt.gz
Exported open-targets-genetics-dev.rsID_sumstats.GCST002245 to gs://genetics-portal-dev-analysis/xg1/rsid_sumstats/GCST002245.txt.gz


In [15]:
from google.cloud import storage

ImportError: cannot import name 'storage' from 'google.cloud' (unknown location)

In [13]:
from google.cloud import bigquery

client = bigquery.Client()

# Perform a query.
QUERY = (
    'SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` '
    'WHERE state = "TX" '
    'LIMIT 100')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.name)

Frances
Alice
Beatrice
Ella
Gertrude
Josephine
Lula
Blanche
Marjorie
Christine
Margarita
Alta
Sara
Naomi
Sofia
Andrea
Delfina
Dominga
Elnora
Adele
Rafaela
Dixie
Luisa
Bess
Ernestine
Lorine
Patsy
Cecelia
Enriqueta
Lucinda
Alyce
Oneta
Apolonia
Gloria
Rhoda
Buna
Maye
Pansy
Gladys
Maria
Helen
Velma
Clara
Anna
Evelyn
Rosa
Mae
Bernice
Stella
Vivian
Mable
Maggie
Betty
Guadalupe
Lorene
Susie
Sadie
Erma
Verna
Amelia
Estelle
Nell
Margie
Celia
Lessie
Eloise
Millie
Olga
Letha
Rachel
Luz
Gussie
Sylvia
Aline
Barbara
Iris
Leila
Mozelle
Simona
Catalina
Hester
Willia
Allene
Avis
George
Isabella
Polly
Syble
Teodora
Lennie
Ricarda
Valerie
Frances
Gertrude
Sarah
Rosa
Lucy
Nora
Nettie
Odessa


In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

query = """
    SELECT name, SUM(number) as total_people
    FROM `bigquery-public-data.usa_names.usa_1910_2013`
    WHERE state = 'TX'
    GROUP BY name, state
    ORDER BY total_people DESC
    LIMIT 20
"""
query_job = client.query(query)  # Make an API request.

print("The query data:")
for row in query_job:
    # Row values can be accessed by field name or index.
    print("name={}, count={}".format(row[0], row["total_people"]))